# CRUD
This notebook demonstrated basic CRUD (Create, Read, Update, Delete) operations for SQLAlchemy Core.

Core means using table definitions, and *not* using the ORM.

A lot of queries are different in the ORM syntax.<br>
When using SQLAlchemy mainly for the ORM, it might be best to learn the ORM before Core.<br>
Do not confuse yourself by reading this notebook first.<br>
**However**, any sufficiently complex query will inevitably need to use Core features.

- Tables
- Transactions
- Insert
  - Returning
- Select
  - Scalar(s)
  - Order By
  - Offset/Limit
  - Where
  - Transformation
  - Aggregate/Group By
  - Union
- Update
- Delete

> For better readability, consider turning on "Table of Content" or "Show Line Numbers" in the 'View' menu.

## Logging

In [ ]:
from contextlib import contextmanager
import logging
import sys

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.WARN)

logger = logging.getLogger('sqlalchemy.engine')
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

@contextmanager
def logs(level=logging.INFO):
    state = handler.level
    handler.setLevel(level)
    try:
        yield
    finally:
        handler.setLevel(state)

## Tables
Although SQLAlchemy is perfectly capable of running *raw* text queries, it can also use Table definitions.

Table definitions serve a few purposes:
1. Simplify renaming/refactoring of table names and column definitions.
2. Creating a table in live databases, or specifically for testing.

These table definitions are set up by creating instancing of the Table class.<br>
Tables will be sharing a MetaData object. This metadata is used to describe the database in one way or another.<br>
For servers with multi-database setups, this metadata can allow a single network connection to be used for multiple database at the same time.

The signature of the constructor is roughly this (slightly edited for clarity).
```python
class Table(...):
    def __init__(self, tablename: str, metadata: MetaData, *columns: Column, **kwargs):
        ...
```
It is also possible to create table definitions which do not use a MetaData object.<br>
For the purposes of an introduction, this will be introduced only in later notebooks.

In [ ]:
import sqlalchemy as sa

In [ ]:
metadata = sa.MetaData()
Products = sa.Table('products', metadata, 
                    sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
                    sa.Column('name', sa.VARCHAR(255), default=None, nullable=True),
                    sa.Column('category', sa.VARCHAR(255), default=None, nullable=True)
                    )

In [ ]:
print('Table:', Products.name)
for column in Products.columns:
    print(f'> {column.key:<10s} NULL={str(column.nullable):<8s}  PK={str(column.primary_key):<8s} {str(column.type):<10s}')

-----
Creating tables in this way can feel chaotic or unmanaged.<br>
The usage of a capital 'P' in the variable name was intentional.<br>
In later notebooks we might use 'product' or 'products' in lowercase to specify a number of items from the Products table.

In [ ]:
engine = sa.create_engine('sqlite://')
con = engine.connect()

In [ ]:
# This is where we actually create the tables.
# The Metadata understands the tables, and giving it the engine to do so.
# Running this multiple times will not recreate tables (data will stay).
metadata.create_all(engine)

## Really Basic
Here are a few 'raw' queries to demonstrate that everything is in working order.

In [ ]:
# Raw Insert
result = con.execute(sa.text('INSERT INTO products(name)VALUES("jack");'))
print(result.rowcount)

In [ ]:
# Raw Select
result = con.execute(sa.text('SELECT * FROM products;'))
for row in result:
    print(row)

In [ ]:
# This is how to use a Table for this.
result = con.execute(Products.select())
for row in result:
    print(row)

## Transactions
It's important to talk about Transactions because SQLAlchemy uses them a lot, and a lack of understanding will make bugs very difficult to solve.

Database transactions are sets of database operations (select, read, update, etc.) which should be applied or reverted as a whole.<br>
If one operation causes an error, all the preceding operation should be reverted.

Remember these things:
1. Other database connections cannot read from your transactions unless they're doing it on purpose.
2. Transactions tend to lock database rows, which can make things slow.
3. SQLAlchemy will try to start transactions every chance it gets. Event when it's just reading, it will start a transaction implicitly.

----------
To start a transaction properly, try calling `.begin()`.

```python
with connection.begin() as transaction:
    ...
```

Transactions *should* automatically commit, unless an exception breaks the context.<br>
This notebook uses a lot rollbacks to keep the demos clean.

```python
print(connection.in_transaction())  # False
print(sa.inspect(connection).get_table_names())
print(connection.in_transaction())  # True
```


In [ ]:
# Quick check-up, also demonstrating how one might need to diagnose things.
if con.closed:
    print("Connection is closed.")
if con.in_transaction():
    print("Connection is in transaction.")
    con.rollback()
    print('Rolled back, changes discarded')

### InvalidRequestError: A common Exception
Calling `begin()` while a transaction is already running will provide the following exception:

> InvalidRequestError: This connection has already initialized a SQLAlchemy Transaction() object via begin() or autobegin; can't call begin() here unless rollback() or commit() is called first.

This usually happens for one of two reasons:
1. Something is using `with engine.begin()`, which starts both a connection and a transaction. (Generally a bad idea).
2. A piece of code dirtied the connection (`sqlalchemy.inspect(...).whatever()` can be the cause of this).

If it's on the connection, this can be solved by using `connection.commit()` or `connection.rollback()`.

## Insert
There are many ways to insert a record.
They're similar, but not quite the same.
When a record has been inserted, SQLAlchemy will return the primary key that has been created/calculated (when applicable).

In [ ]:
# Passing the new row as a dictionary.
result = con.execute(Products.insert(), {'name': 'Record 1'})
print(f'Inserted {result.rowcount:d} row(s).')
if con.in_transaction():
    print('commit!')
    con.commit()

In [ ]:
with con.begin() as transaction:
    # Connection.execute(query, parameters)
    result = con.execute(Products.insert(), {'name': 'Record 2'})
    print(f'Inserted {result.rowcount:d} row(s).')
    transaction.rollback()

with con.begin() as transaction:
    # Prepared object by using `.values()`
    query = sa.insert(Products).values({'name': 'Record 3'})
    con.execute(query)
    print(f'Inserted {result.rowcount:d} row(s).')
    transaction.rollback()
    

----------
**Primary Keys:** When data gets inserted into a table, it's *possible* that to get the newly created primary key.

A key thing to note is bulk inserts.
Any time that multiple rows are inserted, the return value will usually omit the inserted primary key.

**PostgreSQL** can still return the inserted keys in the expected way.
This doesn't always apply when multiple records are inserted, but systems like Postgres do support it.

In [ ]:
with con.begin() as t:
    result = con.execute(Products.insert(), {'name': 'Record 2'})
    print(f'Inserted {result.rowcount:d} row(s)')
    print('Newly created primary key:', result.inserted_primary_key)  # yes, that's a tuple.
    t.rollback()

----------
The `inserted_primary_key` (phrased in singular) is returning a tuple, which is correct.

Relational Database Systems can have a primary key consisting of multiple fields.<br>
SQLAlchemy accounts for this by always returning every inserted key as a tuple.

Things change a bit when inserting multiple records:

In [ ]:
with con.begin() as t:
    # Multiple Inserts
    as_list = [{'name': 'Record 3'}, {'name': 'Record 4'}]
    result = con.execute(Products.insert(), as_list)
    print(f'Inserted {result.rowcount} records')
    print(result.inserted_primary_key_rows)

    # SQLite and others will return a list of empty tuples.
    as_list = [{'name': 'Record 5'},]
    result = con.execute(Products.insert(), as_list)
    print(f'Inserted {result.rowcount} records')
    print(result.inserted_primary_key_rows)
    
    t.rollback()

----------
The above demonstrates that multi-row inserts will not return primary keys.<br>
If this feature is needed, look at the `returning` feature in the next notebook file.

Although the `execute` call  is effectively the same (both passing a list), it is the number of records that determines wether or not a primary key is returned.

**Note:** When writing tests, it's important to remember this detail and write for multiple records if that is a possible situation.
Otherwise the code might be tested on the assumption that primary keys are always returned.

## Select
This is the bread and butter of most queries.

Queries can be built using Table or Column objects.
This query is fed into an `.execute` function, which returns a Result object as usual.

In [ ]:
# Querying for ALL Columns
query = Products.select()

print('SQL:', str(query).replace('\n', ''))
print('All columns are spelled out by SQLAlchemy, it will not use a wildcard when using this syntax.\n')

result = con.execute(query)
print(result)  # CursorResult
for row in result:  # Row (tuple-styled)
    print(row, type(row))

print('---')

result = con.execute(query).mappings()  
print(type(result))  # MappingResult
for row in result:  # RowMapping (dict-styled)
    print(row, type(row))


In [ ]:
# Selecting specific columns
# The columns contain metadata about the table(s) to query.
# Note that '.columns' and '.c' are the same thing, it is just a writing aid.
query = sa.select(Products.c.id, Products.c.name)

# These can be accessed using the bracket syntax as well. 
query = sa.select(Products.c['id'], Products.c['name'])
# unpack it like a tuple (it stays in order).
for pk, name in con.execute(query):
    print(pk, name)

----------
The above shows rows as tuples and dictionaries when printed.<br>
The underlying object is usually a `Row` or `RowMapping` object.

These allow array-like access with some extras.

**Example:** Queries sometimes get created dynamically, and that includes columns.<br>
The system that adds columns also has to read them. Doing this by a textual key can be a bit iffy.<br>


In [ ]:
# After using .mappings(), fields can be access using column definitions.
# This comes in handy for 'calculated columns' later on.
column_id = Products.c.id
column_name = Products.c.name

query = sa.select(column_id, column_name)
for entry in con.execute(query).mappings():
    print(entry[column_id], entry[column_name])

In [ ]:
query = sa.select(Products.c.id)
print('A:', query)
query = query.add_columns(Products.c.name)
print('B:', query)

In [ ]:
# Appending `scalars()` will return the first column for each row.
# The query will still fetch the entire set.
for pk in con.execute(Products.select()).scalars():
    print(pk)

In [ ]:
# Appending `scalar()` will return the first column of the first row.
# The query will still fetch the entire set.
pk = con.execute(Products.select()).scalar()
print(pk)

In [ ]:
# In order to select individual columns, `sa.select(*columns)` is used.
query = sa.select(Products.columns.name)
for entry in con.execute(query).mappings():
    print(entry)

### Scalar(s)
The 'Scalar' is the value of the first column.<br>
In SQLAlchemy, the 'scalar' (singular) returns the first column of the first row (any additional data becomes inaccessible).<br>
When using 'scalars' (plural) it returns the first column of *every* row.

Note that all the data still transfered, and gets reduced to a singular column.<br>
The example below has logs turned on to show the query being executed.


In [ ]:
with logs():
    query = sa.select(Products)
    print(con.scalar(query))

### Order By

In [ ]:
# Order by Name, descending:
str(Products.select().order_by(Products.c.name.desc()))

### Offset/Limit

In [ ]:
str(Products.select().offset(10).limit(10))

### Where

Note: Using subqueries and joins are shown in their own notebook.

In [ ]:
query = sa.select(Products).where(Products.columns.id <= 2)
for product in con.execute(query):
    print(product)

In [ ]:
query = sa.select(Products).order_by(Products.columns.id).offset(1).limit(1)

In [ ]:
with logs():
    for product in con.execute(query):
        print(product)

In [ ]:
query = sa.select(Products).where(Products.columns.id.in_([4,5,6]))

In [ ]:
with logs():
    for product in con.execute(query):
        print(product)

### Transformation

In [ ]:
query = sa.select(Products.columns.id.label('double'))

In [ ]:
for product in con.execute(query).mappings():
    print(product)

### Union

Remember that `UNION` will deduplicate results, whereas `UNION ALL` will not.

In [ ]:
query_a = sa.select(Products.columns.id.label('x')).where(Products.columns.id == 1)
query_b = sa.select(Products.columns.id.label('x')).where(Products.columns.id == 1)

with logs():
    for product in con.execute(sa.union_all(query_a, query_b)).all():
        print(product)

## Update

In [ ]:
# The preceding 'select' section did not use transactions, so it might be dirty.
if con.in_transaction():
    con.rollback()

In [ ]:
with con.begin():
    for row in con.execute(Products.select()):
        print(row)

In [ ]:
with con.begin() as t:
    query = (
        Products.update()
        .where(Products.c.id == 1)
        .values(name='Record X')
    )
    con.execute(query)
    for row in con.execute(Products.select()):
        print(row)    
    t.rollback()

## Delete

In [ ]:
query_count = sa.select(sa.func.count(Products.c.name).label('count'))
with con.begin() as t:
    r = con.scalar(query_count)
    print(r)

In [ ]:
with con.begin() as t:
    result = con.execute(Products.delete())
    print(f'deleted {result.rowcount} rows')
    print(f'{con.scalar(query_count)} rows in database')
    t.rollback()